<a href="https://colab.research.google.com/github/hiteshJindal/Thesis_Audio_Uncertainity/blob/main/Main_thesis_file_without_spectograms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydub

In [ ]:
import pandas as pd
from pydub.utils import mediainfo
from pydub import AudioSegment

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

# Set the audio directory
#audio_dir = '/gdrive/MyDrive/Audio_Dataset/textgrid/common_voice_en_33170883.TextGrid'

In [ ]:
import os
# Create an empty DataFrame to store the results
final_df = pd.DataFrame(columns=['Transcript', 'phoneme_likelihood', 'Phones'])

directory = '/gdrive/MyDrive/Input_large_final/Input_large/senddrive/Textgrid_done_big_wav_transcription/'


def parse_textgrid(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = []
    start_time, end_time, label = None, None, None
    for line in lines:
        line = line.strip()
        if line.startswith('xmin'):
            start_time = float(line.split('=')[1].strip())
        elif line.startswith('xmax'):
            end_time = float(line.split('=')[1].strip())
        elif line.startswith('text'):
            label = line.split('=')[1].strip().strip('"')
            if start_time is not None and end_time is not None and label is not None:
                data.append((start_time, end_time, label))
                start_time, end_time, label = None, None, None

    return data

def textgrid_to_dataframe(file_path):
    data = parse_textgrid(file_path)
    df = pd.DataFrame(data, columns=['Start Time', 'End Time', 'Label'])
    return df


# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.TextGrid'):
        file_path = os.path.join(directory, filename)

        # Process the file and obtain the necessary dataframes
        df = textgrid_to_dataframe(file_path)

        # Get the indices of the matched rows
        indices = df.index[(df['Start Time'] == df['Start Time'].iloc[0]) & (df['End Time'] == df['End Time'].iloc[0])]

        # Split the DataFrame based on indices
        first_df = df.loc[:indices[-1]]
        second_df = df.loc[indices[-1]+1:]

        # Remove rows with blank or null labels from first_df
        first_df = first_df[first_df['Label'].notnull() & (first_df['Label'] != "")]

        # Remove rows with blank or null labels from second_df
        second_df = second_df[second_df['Label'].notnull() & (second_df['Label'] != "")]

        # Combine labels from first_df into a single sentence
        combined_sentence = ' '.join(first_df['Label'].tolist())

        # Create Combined_df with the combined sentence
        combined_df = pd.DataFrame({'Transcript': [combined_sentence]})

        # Find the highest occurring string in second_df
        phoneme_likelihood = second_df['Label'].mode().iloc[0]

        # Create Transcript DataFrame with the highest occurring string
        transcript_df = pd.DataFrame({'phoneme_likelihood': [phoneme_likelihood]})

        # Combine labels from second_df into a list
        phones_list = second_df['Label'].tolist()

        # Create Phones DataFrame with the list of phones
        phones_df = pd.DataFrame({'Phones': [phones_list]})

        # Concatenate the DataFrames and append to the final_df
        result_df = pd.concat([combined_df, transcript_df, phones_df], axis=1)
        final_df = pd.concat([final_df, result_df], ignore_index=True)



In [ ]:
import random
# Create an empty DataFrame to store the results
final_test_df = pd.DataFrame(columns=['Transcript', 'phoneme_likelihood', 'Phones'])

directory = '/gdrive/MyDrive/Input_large_final/Input_large/senddrive/wav_textgrid_big_test/'

def parse_textgrid(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    data = []
    start_time, end_time, label = None, None, None
    for line in lines:
        line = line.strip()
        if line.startswith('xmin'):
            start_time = float(line.split('=')[1].strip())
        elif line.startswith('xmax'):
            end_time = float(line.split('=')[1].strip())
        elif line.startswith('text'):
            label = line.split('=')[1].strip().strip('"')
            if start_time is not None and end_time is not None and label is not None:
                data.append((start_time, end_time, label))
                start_time, end_time, label = None, None, None

    return data

def textgrid_to_dataframe(file_path):
    data = parse_textgrid(file_path)
    df = pd.DataFrame(data, columns=['Start Time', 'End Time', 'Label'])
    return df

# Iterate over the files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.TextGrid'):
        file_path = os.path.join(directory, filename)

        # Process the file and obtain the necessary dataframes
        df = textgrid_to_dataframe(file_path)

        # Get the indices of the matched rows
        indices = df.index[(df['Start Time'] == df['Start Time'].iloc[0]) & (df['End Time'] == df['End Time'].iloc[0])]

        # Split the DataFrame based on indices
        first_test_df = df.loc[:indices[-1]]
        second_test_df = df.loc[indices[-1]+1:]

        # Remove rows with blank or null labels from first_df
        first_test_df = first_test_df[first_test_df['Label'].notnull() & (first_test_df['Label'] != "")]

        # Remove rows with blank or null labels from second_df
        second_test_df = second_test_df[second_test_df['Label'].notnull() & (second_test_df['Label'] != "")]

        # Combine labels from first_df into a single sentence
        combined_sentence_test = ' '.join(first_test_df['Label'].tolist())

        # Create Combined_df with the combined sentence
        combined_test_df = pd.DataFrame({'Transcript': [combined_sentence_test]})

        # Find the highest occurring strings in second_df
        highest_occurrences = second_test_df['Label'].mode()

        if not highest_occurrences.empty:
            # Choose a random element from the list of highest occurrences
            phoneme_likelihood_test = random.choice(highest_occurrences.tolist())
        else:
            phoneme_likelihood_test = None  # Handle the case where there are no labels

        # Create Transcript DataFrame with the highest occurring string
        transcript_test_df = pd.DataFrame({'phoneme_likelihood': [phoneme_likelihood_test]})

        # Create Phones DataFrame with the list of phones
        phones_test_df = pd.DataFrame({'Phones': [second_test_df['Label'].tolist()]})

        # Concatenate the DataFrames and append to the final_df
        result_test_df = pd.concat([combined_test_df, transcript_test_df, phones_test_df], axis=1)
        final_test_df = pd.concat([final_test_df, result_test_df], ignore_index=True)


In [ ]:
print(final_test_df.shape)
print(final_df.shape)

In [ ]:
final_df.head(150)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Flatten, BatchNormalization, MaxPooling1D, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dropout, Dense, LSTM, Reshape
import tensorflow as tf

# Example with L2 regularization for dense layers

# Extract the input features and response variable
X = final_df['Transcript']
y = final_df['phoneme_likelihood']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to have the same length
max_length = max(max(len(seq) for seq in X_train_seq), max(len(seq) for seq in X_test_seq))
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Encode the response variable
label_encoder = LabelEncoder()
label_encoder.fit(y)  # Fit the label encoder on all labels in y
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Define the improved model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(BatchNormalization())
model.add(Dropout(0.5))
# Add another Conv1D layer
model.add(Dense(units=64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Define the model checkpoint callback to save the best model during training

# Train the model
history = model.fit(X_train_padded, y_train_encoded, epochs=20, batch_size=128, validation_data=(X_test_padded, y_test_encoded))


# Calculate training and validation accuracy
train_accuracy = model.evaluate(X_train_padded, y_train_encoded)[1]
val_accuracy = model.evaluate(X_test_padded, y_test_encoded)[1]
print("Training Accuracy:", train_accuracy)
print("Validation Accuracy:", val_accuracy)

In [ ]:
# Assume you have a new DataFrame called 'new_data' with the same column names as 'final_df'

# Extract the input features from the new data
X_new = final_test_df['Transcript']

# Convert text to sequences using the tokenizer fitted on the training data
X_new_seq = tokenizer.texts_to_sequences(X_new)

# Pad sequences to have the same length as the training data
X_new_padded = pad_sequences(X_new_seq, maxlen=max_length, padding='post')

# Make predictions on the new data
predictions = model.predict(X_new_padded)

# Decode the predicted labels
predicted_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=1))
print(predicted_labels)
print(final_test_df['phoneme_likelihood'])
# Calculate the accuracy on the new data
accuracy = np.mean(predicted_labels == final_test_df['phoneme_likelihood'])

print("Accuracy on New Data:", accuracy)


In [ ]:
print(np.sum(predicted_labels == final_test_df['phoneme_likelihood']))